In [927]:
import pandas as pd
import warnings
import numpy as np
import re
import math

warnings.filterwarnings("ignore")

In [928]:
df = pd.read_csv("data_fire_18.csv")
df

,Timestamp,Which GROUP does the student belong to?,Student's Full Name,Student's Date of Birth (DOB),Student's Gender,District the student belongs to.,Name of the Samithi.,Student's Year of Joining Balvikas,Student's Date of Joining Balvikas (Optional),Has the student passed Group 2 exam?,...,Number of accompanying Mahilas(Women) who will need accommodation,Any food allergies for student? (Optional),Check In Date,Check In Time,Check out Date,Check out Time,Valid Group Selected based on DOB?,Valid Events Registered?,Overall Registration Status,Remarks
0,11/5/2024 13:45:27,Group 2,SAMYUTHA SRIRAM,"May 3, 2013",Female,Chennai South,Alwarpet,2022,"June 14, 2022",NaN,...,1.0,NaN,"December 25, 2024",6 AM,"December 25, 2024",830 PM,Accepted,Accepted,Accepted,
1,11/7/2024 22:24:42,Group 1,ANIRUDH SAI GUHAN .R,"March 19, 2016",Male,Coimbatore,Podanur,2020,"November 13, 2020",NaN,...,0.0,No,"December 25, 2024",6.30am,"December 25, 2024",5pm,Accepted,Accepted,Accepted,
2,11/8/2024 16:07:39,Group 1,G.LAKSHITH,"June 7, 2016",Male,Coimbatore,Saibaba colony,2022,"June 30, 2022",NaN,...,0.0,No,"December 25, 2024",6.30am,"December 25, 2024",5pm,Accepted,Accepted,Accepted,
3,11/8/2024 21:10:59,Group 1,SAI CHARANYA SRI P.S,"April 16, 2016",Female,Coimbatore,Podanur,2020,"June 5, 2020",NaN,...,0.0,NaN,"December 25, 2024",6.30 am,"December 25, 2024",5pm,Accepted,Accepted,Accepted,
4,11/10/2024 22:42:34,Group 2,S.SAI ESHWAR,"October 24, 2013",Male,Tiruvannamalai,Tiruvannamalai main samithi,2023,"June 18, 2023",NaN,...,1.0,No,"December 24, 2024",11 pm,"December 25, 2024",6:00pm,Accepted,Accepted,Accepted,
...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...
774,12/12/2024 15:12:44,Group 2,V VEDASHREE,"January 3, 2015",Female,Chennai North West,ASHTALAKSHMI NAGAR,2021,"November 20, 2021",NaN,...,1.0,NO,"December 24, 2024",NaN,"December 25, 2024",NaN,Accepted,Accepted,Accepted,
775,12/12/2024 15:12:45,Group 2,KANISHA B.M.,"January 24, 2013",Female,Chennai North West,GILL NAGAR,2022,"July 9, 2022",NaN,...,2.0,NO,"December 24, 2024",3.00PM,"December 25, 2024",9PM,Accepted,Accepted,Accepted,
776,12/12/2024 15:12:46,Group 2,ROSHAN SARAVANA,"October 7, 2013",Male,Chennai North West,ANNA NAGAR EAST,2019,"November 30, 2019",NaN,...,1.0,NO,"December 24, 2024",3.00PM,"December 25, 2024",9PM,Accepted,Accepted,Accepted,
777,12/12/2024 15:12:47,Group 2,GAYATHRI R S,"March 25, 2014",Female,Chennai North West,SAI SAHARA,2021,"January 24, 2021",NaN,...,NaN,NO,"December 24, 2024",7.00AM,"December 25, 2024",9PM,Accepted,Accepted,Accepted,


In [929]:
def formatModeOfTravel(val):
    val = str(val)
    if "train" in val.lower() or "traom" in val.lower() or "teajn" in val.lower():
        return "Train"
    if "bus" in val.lower():
        return "Bus"
    if "car" in val.lower():
        return "Car"
    if "van" in val.lower():
        return "Van"
    if "flight" in val.lower():
        return "Flight"
    if "own" in val.lower():
        return "Self"
    if "no" in val.lower():
        return "-"
    if "KOVAI" in val:
        return "Train"

df["Mode of Travel"] = df["Mode of Travel"].apply(formatModeOfTravel)
df["Mode of Travel"].value_counts()

Mode of Travel
Train     432
Bus       123
Car        41
Van        33
Self       29
Flight      2
-           1
Name: count, dtype: int64

In [930]:
df["Mode of Travel.1"] = df["Mode of Travel.1"].apply(formatModeOfTravel)
df["Mode of Travel.1"].value_counts()

Mode of Travel.1
Train     379
Bus       155
Car        41
Van        31
Self       30
-           3
Flight      1
Name: count, dtype: int64

In [931]:
# Function to clean and convert time values to HH:MM <AM/PM> format
def standardize_time_format(time_str):
    try:
        # Handle missing or invalid values
        if pd.isnull(time_str) or not isinstance(time_str, str) or not time_str.strip():
            return ''

        # Normalize input by removing extra spaces and converting to lowercase
        time_str = time_str.strip().lower()

        time_str = re.sub(r'[\s,;]+', ':', time_str) 
        time_str = re.sub(r'::+', ':', time_str)      

        # Specific case handling
        if "noon" in time_str:
            return "12:00 PM"
        if "midnight" in time_str:
            return "12:00 AM"
            
        time_str = re.sub(r'^(\d{1,2})(am|pm)$', r'\1:00 \2', time_str) 
        time_str = re.sub(r'^(\d{1,2}):(\d{2})(am|pm)$', r'\1:\2 \3', time_str)  
        time_str = re.sub(r'^(\d{1,2})\s+(am|pm)$', r'\1:00 \2', time_str)  
        time_str = re.sub(r'^(\d{1,2}):(\d{2})\s?(am|pm)$', r'\1:\2 \3', time_str)  
        time_str = re.sub(r'^(\d{1,2}):\s?(am|pm)$', r'\1:00 \2', time_str)  
        time_str = re.sub(r'^(\d{1,2})m$', r'\1:00 am', time_str) 
        time_str = re.sub(r'^(\d{1,2}):(\d{2}):?(am|pm)$', r'\1:\2 \3', time_str)  
        time_str = re.sub(r'^(\d{1,2})\.(\d{2})\s?(am|pm)$', r'\1:\2 \3', time_str) 
        time_str = re.sub(r'^(\d{1,2})\.(\d{2}):?(am|pm)$', r'\1:\2 \3', time_str)  
        time_str = re.sub(r'^(\d{1,2})\.(am|pm)$', r'\1:00 \2', time_str)  
        time_str = re.sub(r'^(\d{1,2})\.(\d{2}):?a\.m\.?$', r'\1:\2 am', time_str) 
        time_str = re.sub(r'^(\d{1,2})\.:a\.m\.?$', r'\1:00 am', time_str)
        time_str = re.sub(r'^0?(\d{1,2})\.(\d{2})\.?:am$', r'\1:\2 am', time_str)
        time_str = re.sub(r'^(\d{1,2})\.(\d{2}):p\.m$', r'\1:\2 pm', time_str)
        time_str = re.sub(r'^(\d{1,2})(\d{2}):pm$', r'\1:\2 pm', time_str)
        time_str = re.sub(r'^(\d{1,2})\.(\d{2}):p\.m\.?$', r'\1:\2 pm', time_str)
        time_str = re.sub(r'^(\d{1,2})\.p\.m$', r'\1:00 pm', time_str)
        time_str = re.sub(r'^(2[0-3]|[01]?\d)\.(\d{2}):hrs$', lambda m: f"{int(m.group(1)) % 12 or 12}:{m.group(2)} {'pm' if int(m.group(1)) >= 12 else 'am'}", time_str)
        time_str = re.sub(r'^(\d{1,2}):a\.m\.?$', r'\1:00 am', time_str)
        time_str = re.sub(r'^(\d{1,2})\.(\d{2})mpm$', r'\1:\2 pm', time_str)
        time_str = re.sub(r'^0?(\d{1,2})\.?:00:pm$', r'\1:00 pm', time_str)
        time_str = re.sub(r'^nil$', '', time_str)
        time_str = re.sub(r'^(\d{1,2})\.(\d{2})$', r'\1:\2 pm', time_str)
        time_str = re.sub(r'^0?(\d{1,2}):(\d{2})$', lambda m: f"{int(m.group(1)) % 12 or 12}:{m.group(2)} {'pm' if int(m.group(1)) >= 12 else 'am'}", time_str)
        time_str = re.sub(r'^(\d{1,2})\.(\d{2})p\.m$', r'\1:\2 pm', time_str)
        time_str = re.sub(r'^(\d{1,2})p\.m\.?$', r'\1:00 pm', time_str)
        time_str = re.sub(r'^(\d{1,2})\.(\d{2})p\.m$', r'\1:\2 pm', time_str)
        time_str = re.sub(r'^(\d{1,2}):p\.m\.?$', r'\1:00 pm', time_str)
        time_str = re.sub(r'^(\d{1,2}):p\.m\.?$', r'\1:00 pm', time_str)
        time_str = re.sub(r'^0?(\d{1,2}):p\.m\.?$', r'\1:00 pm', time_str)
        time_str = re.sub(r'^(\d{1,2}):p\.m\.?$', r'\1:00 pm', time_str)
        time_str = re.sub(r'^(\d{1,2})\.(\d{2})p\.m$', r'\1:\2 pm', time_str)
 
        match = re.match(r'^(\d{1,2}):(\d{2})\s*(am|pm)$', time_str)
        if not match:
            print(f"Invalid time format: {time_str}")
            return ''

        hour, minute, period = match.groups()
        hour = int(hour)
        minute = int(minute)

        if hour < 1 or hour > 12 or minute < 0 or minute >= 60:
            print(f"Out of range time: {time_str}")
            return ''
        # Format time as HH:MM <AM/PM>
        formatted_time = f"{hour}:{minute:02d} {period.upper()}"
        return formatted_time
    except Exception as e:
        print(f"Error processing {time_str}: {e}")
        return ''

def unifyLength(val):
    val = str(val)
    val = val.strip()
    if len(val) != 0 and len(val) != 8:
        val = "0" + val
    return val

# Columns to process
time_columns = ['Check In Time', 'Check out Time', 'Time of arrival', 'Time of departure']
# Apply the transformation
for col in time_columns:
    if col in df.columns:
        df[col] = df[col].apply(standardize_time_format)
        df[col] = df[col].apply(unifyLength)

Invalid time format: 


In [932]:
for time_column in time_columns:
    print(f"Unique values in {time_column} column:")
    x = df[time_column].unique()
    x.sort()
    print(x)

Unique values in Check In Time column:
['' '01:00 PM' '02:00 PM' '02:05 AM' '02:05 PM' '02:15 PM' '02:30 PM'
 '03:00 PM' '04:00 AM' '04:00 PM' '05:00 AM' '05:00 PM' '05:10 AM'
 '05:30 AM' '06:00 AM' '06:00 PM' '06:30 AM' '06:30 PM' '07:00 AM'
 '07:00 PM' '07:15 AM' '07:30 AM' '07:30 PM' '07:45 AM' '08:00 AM'
 '08:00 PM' '08:15 PM' '08:30 PM' '09:00 AM' '09:00 PM' '09:27 PM'
 '09:30 PM' '09:45 AM' '09:45 PM' '10:00 AM' '10:00 PM' '10:15 PM'
 '10:20 PM' '10:30 AM' '10:30 PM' '11:00 AM' '11:00 PM' '12:00 PM'
 '12:30 PM']
Unique values in Check out Time column:
['' '01:00 PM' '02:00 PM' '02:15 PM' '03:00 PM' '03:15 PM' '04:00 PM'
 '04:30 AM' '04:30 PM' '05:00 AM' '05:00 PM' '05:15 AM' '05:30 AM'
 '05:30 PM' '06:00 AM' '06:00 PM' '06:20 AM' '06:30 PM' '07:00 AM'
 '07:00 PM' '08:00 PM' '08:30 AM' '08:30 PM' '08:45 PM' '09:00 PM'
 '09:15 PM' '09:30 PM' '09:45 PM' '10:00 AM' '10:00 PM' '10:20 PM'
 '10:25 PM' '10:50 PM' '11:00 AM' '11:00 PM' '11:45 PM' '12:00 PM']
Unique values in Time of arriv

In [933]:
# District

# df["District the student belongs to."].unique()

district = [
    "Chennai South",
    "Coimbatore",
    "Tiruvannamalai",
    "Tirunelveli",
    "Kanchipuram North",
    "Thanjavur",
    "Chennai South East",
    "Kanchipuram South",
    "Madurai",
    "Virudhunagar",
    "Trichy",
    "Dharmapuri / Krishnagiri",
    "Cuddalore",
    "Kanyakumari",
    "Chennai West",
    "Salem",
    "Nilgiris",
    "Tiruvallur East",
    "Tuticorin",
    "Tirupur",
    "Chennai North",
    "Sivaganga&Ramnad",
    "Erode",
    "Chennai North West",
    "Puducherry",
    "Vellore",
    "Dindigul",
    "Chennai East Coast",
    "Karur",
    "Theni",
    "Mayiladuthurai",
    "Nagapattinam",
    "Villupuram"
]

district_code = {}

for i in range(len(district)):
    district_code[district[i]] = str(i + 1).zfill(2)

In [934]:
def processData(df):
    df["Timestamp"] = pd.to_datetime(df["Timestamp"])
    df = df.sort_values(by="Timestamp")
    df = df.reset_index(drop=True)
    df = df.drop(
        [
            "Timestamp",
            "Valid Group Selected based on DOB?",
            "Valid Events Registered?",
            # "Remarks",
        ],
        axis=1,
    )
    column_mapping = {
        "Which GROUP does the student belong to?": "studentGroup",
        "Student's Full Name": "studentFullName",
        "Student's Date of Birth (DOB)": "dateOfBirth",
        "Student's Gender": "gender",
        "District the student belongs to.": "district",
        "Name of the Samithi.": "samithiName",
        "Student's Year of Joining Balvikas": "yearOfJoiningBalvikas",
        "Student's Date of Joining Balvikas (Optional)": "dateOfJoiningBalvikas",
        "Has the student passed Group 2 exam?": "hasPassedGroup2Exam",
        "GROUP 1 - Select 1st event to register for.": "group1Event1",
        "GROUP 1 - Select 2nd event to register for. (Optional)": "group1Event2",
        "GROUP 1 - Select the group event to register for. (Optional)": "group1GroupEvent",
        "Select 1st event to register for.": "event1",
        "Select 2nd event to register for. (Optional)": "event2",
        'Select "Yes" to register for Quiz.': "isRegisteredForQuiz",
        "Date of arrival": "arrivalDate",
        "Time of arrival": "arrivalTime",
        "Does the student need pickup facility.": "needsPickup",
        "Mode of Travel": "modeOfTravel",
        "Pick Up Point": "pickupPoint",
        "Date of departure": "departureDate",
        "Time of departure": "departureTime",
        "Student needs drop facility.": "needsDrop",
        "Mode of Travel.1": "modeOfTravelForDrop",
        "Drop Off Point": "dropOffPoint",
        "Are adults accompanying the student?": "hasAccompanyingAdults",
        "Number of Gents(Men) accompanying student": "numMaleAccompanying",
        "Number of Mahilas(Women) accompanying student": "numFemaleAccompanying",
        "Number of non-participating siblings accompanying the student": "numNonParticipatingSiblings",
        "Name of the accompanying person": "accompanyingPersonName",
        "Gender of the accompanying person": "accompanyingPersonGender",
        "Relation": "accompanyingPersonRelation",
        "Contact Number of accompanying person": "accompanyingPersonContact",
        "Age of accompanying person": "accompanyingPersonAge",
        "Does the student need accommodation?": "needsAccommodation",
        "Number of accompanying Gents(Male) who will need accommodation": "numMaleAccompanyingNeedAccommodation",
        "Number of accompanying Mahilas(Women) who will need accommodation": "numFemaleAccompanyingNeedAccommodation",
        "Any food allergies for student? (Optional)": "foodAllergies",
        "Check In Date": "checkInDate",
        "Check In Time": "checkInTime",
        "Check out Date": "checkOutDate",
        "Check out Time": "checkOutTime",
        "Overall Registration Status": "overallRegistrationStatus",
        "Remarks": "remarks",
    }
    df = df.rename(columns=column_mapping)
    return df

group_code = {
    "Group 1": "G1",
    "Group 2": "G2",
    "Group 3": "G3",
    "General Category": "G4",
}

gender_code = {"Male": "M", "Female": "F"}

def prepareData(df):
    df = processData(df)
    data = df.to_dict(orient="records")
    uid = 1

    # Convert nan fields to empty strings
    for i in range(len(data)):
        for key in data[i].keys():
            if (
                key
                in [
                    "group1Event1",
                    "group1Event2",
                    "event1",
                    "event2",
                    "group1GroupEvent",
                ]
                and type(data[i][key]) == float
                and math.isnan(data[i][key])
            ):
                data[i][key] = ""

    for i in range(len(data)):
        if data[i]["studentGroup"] == "Group 1":
            if len(data[i]["group1Event1"]) > 0:
                data[i]["group1Event1"] = f"G1-{data[i]['group1Event1']}"

            if len(data[i]["group1Event2"]) > 0:
                data[i]["group1Event2"] = f"G1-{data[i]['group1Event2']}"

            data[i]["registeredEvents"] = [
                data[i]["group1Event1"],
                data[i]["group1Event2"],
                data[i]["group1GroupEvent"],
            ]

            del data[i]["group1Event1"]
            del data[i]["group1Event2"]
            del data[i]["group1GroupEvent"]
            del data[i]["isRegisteredForQuiz"]
            del data[i]["event1"]
            del data[i]["event2"]
            del data[i]["hasPassedGroup2Exam"]

        elif data[i]["studentGroup"] == "Group 2":
            if "GROUP" not in data[i]["event1"] and len(data[i]["event1"]) > 0:
                data[i]["event1"] = f"G2-{data[i]['event1']}"
            if "GROUP" not in data[i]["event2"] and len(data[i]["event2"]) > 0:
                data[i]["event2"] = f"G2-{data[i]['event2']}"

            data[i]["registeredEvents"] = [data[i]["event1"], data[i]["event2"]]
            if data[i]["isRegisteredForQuiz"] == "Yes":
                data[i]["registeredEvents"].append("Quiz")

            del data[i]["event1"]
            del data[i]["event2"]
            del data[i]["isRegisteredForQuiz"]
            del data[i]["group1Event1"]
            del data[i]["group1Event2"]
            del data[i]["group1GroupEvent"]
            del data[i]["hasPassedGroup2Exam"]

        elif data[i]["studentGroup"] == "Group 3":
            if "GROUP" not in data[i]["event1"] and len(data[i]["event1"]) > 0:
                data[i]["event1"] = f"G3-{data[i]['event1']}"
            if "GROUP" not in data[i]["event2"] and len(data[i]["event2"]) > 0:
                data[i]["event2"] = f"G3-{data[i]['event2']}"

            data[i]["registeredEvents"] = [data[i]["event1"], data[i]["event2"]]
            if data[i]["isRegisteredForQuiz"] == "Yes":
                data[i]["registeredEvents"].append("Quiz")

            del data[i]["event1"]
            del data[i]["event2"]
            del data[i]["isRegisteredForQuiz"]
            del data[i]["group1Event1"]
            del data[i]["group1Event2"]
            del data[i]["group1GroupEvent"]

        else:
            data[i]["registeredEvents"] = []
            if data[i]["isRegisteredForQuiz"] == "Yes":
                data[i]["registeredEvents"].append("Quiz")

            del data[i]["event1"]
            del data[i]["event2"]
            del data[i]["isRegisteredForQuiz"]
            del data[i]["group1Event1"]
            del data[i]["group1Event2"]
            del data[i]["group1GroupEvent"]
            del data[i]["hasPassedGroup2Exam"]

        data[i]["registeredEvents"] = [
            x for x in data[i]["registeredEvents"] if str(x) != ""
        ]

        data[i]["totalParticipatingEvents"] = len(data[i]["registeredEvents"])

        data[i][
            "studentId"
        ] = f"{group_code[data[i]['studentGroup']]}{gender_code[data[i]['gender']]}{district_code[data[i]['district']]}{str(len(data[i]['registeredEvents'])).zfill(2)}{str(uid).zfill(3)}"
        uid += 1
        # print(data[i]['studentId'])

    return data

In [935]:
data = prepareData(df)
new_df = pd.DataFrame(data)
new_df

,studentGroup,studentFullName,dateOfBirth,gender,district,samithiName,yearOfJoiningBalvikas,dateOfJoiningBalvikas,arrivalDate,arrivalTime,...,checkInDate,checkInTime,checkOutDate,checkOutTime,overallRegistrationStatus,remarks,registeredEvents,totalParticipatingEvents,studentId,hasPassedGroup2Exam
0,Group 2,SAMYUTHA SRIRAM,"May 3, 2013",Female,Chennai South,Alwarpet,2022,"June 14, 2022","December 24, 2024",05:00 AM,...,"December 25, 2024",06:00 AM,"December 25, 2024",08:30 PM,Accepted,,[GROUP - Devotional Singing - Girls],1,G2F0101001,NaN
1,Group 1,ANIRUDH SAI GUHAN .R,"March 19, 2016",Male,Coimbatore,Podanur,2020,"November 13, 2020","December 25, 2024",06:30 AM,...,"December 25, 2024",06:30 AM,"December 25, 2024",05:00 PM,Accepted,,[G1-Vedam Chanting],1,G1M0201002,NaN
2,Group 1,G.LAKSHITH,"June 7, 2016",Male,Coimbatore,Saibaba colony,2022,"June 30, 2022","December 25, 2024",06:30 AM,...,"December 25, 2024",06:30 AM,"December 25, 2024",05:00 PM,Accepted,,[G1-Story Telling - Tamizh],1,G1M0201003,NaN
3,Group 1,SAI CHARANYA SRI P.S,"April 16, 2016",Female,Coimbatore,Podanur,2020,"June 5, 2020","December 25, 2024",06:30 AM,...,"December 25, 2024",06:30 AM,"December 25, 2024",05:00 PM,Accepted,,[G1-Story Telling - English],1,G1F0201004,NaN
4,Group 1,S.SAI VAISHNAVI,"August 7, 2018",Female,Tiruvannamalai,Tiruvannamalai main samithi,2023,"June 26, 2023","December 24, 2024",11:00 PM,...,"December 24, 2024",11:00 PM,"December 25, 2024",06:00 PM,Accepted,,"[G1-Vedam Chanting, G1-Drawing]",2,G1F0302005,NaN
...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...
774,Group 2,V VEDASHREE,"January 3, 2015",Female,Chennai North West,ASHTALAKSHMI NAGAR,2021,"November 20, 2021","December 24, 2024",04:00 PM,...,"December 24, 2024",,"December 25, 2024",,Accepted,,[G2-Elocution - Tamizh],1,G2F2401775,NaN
775,Group 2,KANISHA B.M.,"January 24, 2013",Female,Chennai North West,GILL NAGAR,2022,"July 9, 2022","December 24, 2024",02:00 PM,...,"December 24, 2024",03:00 PM,"December 25, 2024",09:00 PM,Accepted,,[G2-Tamizh Chants - Girls],1,G2F2401776,NaN
776,Group 2,ROSHAN SARAVANA,"October 7, 2013",Male,Chennai North West,ANNA NAGAR EAST,2019,"November 30, 2019","December 24, 2024",02:00 PM,...,"December 24, 2024",03:00 PM,"December 25, 2024",09:00 PM,Accepted,,[G2-Drawing],1,G2M2401777,NaN
777,Group 2,GAYATHRI R S,"March 25, 2014",Female,Chennai North West,SAI SAHARA,2021,"January 24, 2021","December 24, 2024",06:00 AM,...,"December 24, 2024",07:00 AM,"December 25, 2024",09:00 PM,Accepted,,[GROUP - Devotional Singing - Girls],1,G2F2401778,NaN


In [936]:
new_df['event1'] = new_df['registeredEvents'].apply(lambda x: x[0] if len(x) > 0 else "")
new_df['event2'] = new_df['registeredEvents'].apply(lambda x: x[1] if len(x) > 1 else "")
new_df.drop(['registeredEvents'], axis=1, inplace=True)
new_df

,studentGroup,studentFullName,dateOfBirth,gender,district,samithiName,yearOfJoiningBalvikas,dateOfJoiningBalvikas,arrivalDate,arrivalTime,...,checkInTime,checkOutDate,checkOutTime,overallRegistrationStatus,remarks,totalParticipatingEvents,studentId,hasPassedGroup2Exam,event1,event2
0,Group 2,SAMYUTHA SRIRAM,"May 3, 2013",Female,Chennai South,Alwarpet,2022,"June 14, 2022","December 24, 2024",05:00 AM,...,06:00 AM,"December 25, 2024",08:30 PM,Accepted,,1,G2F0101001,NaN,GROUP - Devotional Singing - Girls,
1,Group 1,ANIRUDH SAI GUHAN .R,"March 19, 2016",Male,Coimbatore,Podanur,2020,"November 13, 2020","December 25, 2024",06:30 AM,...,06:30 AM,"December 25, 2024",05:00 PM,Accepted,,1,G1M0201002,NaN,G1-Vedam Chanting,
2,Group 1,G.LAKSHITH,"June 7, 2016",Male,Coimbatore,Saibaba colony,2022,"June 30, 2022","December 25, 2024",06:30 AM,...,06:30 AM,"December 25, 2024",05:00 PM,Accepted,,1,G1M0201003,NaN,G1-Story Telling - Tamizh,
3,Group 1,SAI CHARANYA SRI P.S,"April 16, 2016",Female,Coimbatore,Podanur,2020,"June 5, 2020","December 25, 2024",06:30 AM,...,06:30 AM,"December 25, 2024",05:00 PM,Accepted,,1,G1F0201004,NaN,G1-Story Telling - English,
4,Group 1,S.SAI VAISHNAVI,"August 7, 2018",Female,Tiruvannamalai,Tiruvannamalai main samithi,2023,"June 26, 2023","December 24, 2024",11:00 PM,...,11:00 PM,"December 25, 2024",06:00 PM,Accepted,,2,G1F0302005,NaN,G1-Vedam Chanting,G1-Drawing
...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...
774,Group 2,V VEDASHREE,"January 3, 2015",Female,Chennai North West,ASHTALAKSHMI NAGAR,2021,"November 20, 2021","December 24, 2024",04:00 PM,...,,"December 25, 2024",,Accepted,,1,G2F2401775,NaN,G2-Elocution - Tamizh,
775,Group 2,KANISHA B.M.,"January 24, 2013",Female,Chennai North West,GILL NAGAR,2022,"July 9, 2022","December 24, 2024",02:00 PM,...,03:00 PM,"December 25, 2024",09:00 PM,Accepted,,1,G2F2401776,NaN,G2-Tamizh Chants - Girls,
776,Group 2,ROSHAN SARAVANA,"October 7, 2013",Male,Chennai North West,ANNA NAGAR EAST,2019,"November 30, 2019","December 24, 2024",02:00 PM,...,03:00 PM,"December 25, 2024",09:00 PM,Accepted,,1,G2M2401777,NaN,G2-Drawing,
777,Group 2,GAYATHRI R S,"March 25, 2014",Female,Chennai North West,SAI SAHARA,2021,"January 24, 2021","December 24, 2024",06:00 AM,...,07:00 AM,"December 25, 2024",09:00 PM,Accepted,,1,G2F2401778,NaN,GROUP - Devotional Singing - Girls,


In [937]:
new_df = new_df.apply(lambda x: x.str.strip() if x.dtype == "object" else x)
new_df = new_df.replace('', np.nan)
new_df = new_df.replace(np.nan, '-', regex=True)
new_df = new_df.replace({',': ' ', '"': ' ', '/': ' ', '\'': ' '}, regex=True)
new_df

,studentGroup,studentFullName,dateOfBirth,gender,district,samithiName,yearOfJoiningBalvikas,dateOfJoiningBalvikas,arrivalDate,arrivalTime,...,checkInTime,checkOutDate,checkOutTime,overallRegistrationStatus,remarks,totalParticipatingEvents,studentId,hasPassedGroup2Exam,event1,event2
0,Group 2,SAMYUTHA SRIRAM,May 3 2013,Female,Chennai South,Alwarpet,2022,June 14 2022,December 24 2024,05:00 AM,...,06:00 AM,December 25 2024,08:30 PM,Accepted,-,1,G2F0101001,-,GROUP - Devotional Singing - Girls,-
1,Group 1,ANIRUDH SAI GUHAN .R,March 19 2016,Male,Coimbatore,Podanur,2020,November 13 2020,December 25 2024,06:30 AM,...,06:30 AM,December 25 2024,05:00 PM,Accepted,-,1,G1M0201002,-,G1-Vedam Chanting,-
2,Group 1,G.LAKSHITH,June 7 2016,Male,Coimbatore,Saibaba colony,2022,June 30 2022,December 25 2024,06:30 AM,...,06:30 AM,December 25 2024,05:00 PM,Accepted,-,1,G1M0201003,-,G1-Story Telling - Tamizh,-
3,Group 1,SAI CHARANYA SRI P.S,April 16 2016,Female,Coimbatore,Podanur,2020,June 5 2020,December 25 2024,06:30 AM,...,06:30 AM,December 25 2024,05:00 PM,Accepted,-,1,G1F0201004,-,G1-Story Telling - English,-
4,Group 1,S.SAI VAISHNAVI,August 7 2018,Female,Tiruvannamalai,Tiruvannamalai main samithi,2023,June 26 2023,December 24 2024,11:00 PM,...,11:00 PM,December 25 2024,06:00 PM,Accepted,-,2,G1F0302005,-,G1-Vedam Chanting,G1-Drawing
...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...
774,Group 2,V VEDASHREE,January 3 2015,Female,Chennai North West,ASHTALAKSHMI NAGAR,2021,November 20 2021,December 24 2024,04:00 PM,...,-,December 25 2024,-,Accepted,-,1,G2F2401775,-,G2-Elocution - Tamizh,-
775,Group 2,KANISHA B.M.,January 24 2013,Female,Chennai North West,GILL NAGAR,2022,July 9 2022,December 24 2024,02:00 PM,...,03:00 PM,December 25 2024,09:00 PM,Accepted,-,1,G2F2401776,-,G2-Tamizh Chants - Girls,-
776,Group 2,ROSHAN SARAVANA,October 7 2013,Male,Chennai North West,ANNA NAGAR EAST,2019,November 30 2019,December 24 2024,02:00 PM,...,03:00 PM,December 25 2024,09:00 PM,Accepted,-,1,G2M2401777,-,G2-Drawing,-
777,Group 2,GAYATHRI R S,March 25 2014,Female,Chennai North West,SAI SAHARA,2021,January 24 2021,December 24 2024,06:00 AM,...,07:00 AM,December 25 2024,09:00 PM,Accepted,-,1,G2F2401778,-,GROUP - Devotional Singing - Girls,-


In [938]:
# Order columns

"""
['studentGroup', 'studentFullName', 'dateOfBirth', 'gender', 'district',
       'samithiName', 'yearOfJoiningBalvikas', 'dateOfJoiningBalvikas',
       'arrivalDate', 'arrivalTime', 'needsPickup', 'modeOfTravel',
       'pickupPoint', 'departureDate', 'departureTime', 'needsDrop',
       'modeOfTravelForDrop', 'dropOffPoint', 'hasAccompanyingAdults',
       'numMaleAccompanying', 'numFemaleAccompanying',
       'numNonParticipatingSiblings', 'accompanyingPersonName',
       'accompanyingPersonGender', 'accompanyingPersonRelation',
       'accompanyingPersonContact', 'accompanyingPersonAge',
       'needsAccommodation', 'numMaleAccompanyingNeedAccommodation',
       'numFemaleAccompanyingNeedAccommodation', 'foodAllergies',
       'checkInDate', 'checkInTime', 'checkOutDate', 'checkOutTime',
       'overallRegistrationStatus', 'remarks', 'totalParticipatingEvents',
       'studentId', 'hasPassedGroup2Exam', 'event1', 'event2']
"""

print(len(new_df.columns))

new_df = new_df[[
    'studentId',
    'studentGroup',
    'studentFullName',
    'dateOfBirth',
    'gender',
    'district',
    'samithiName',
    'yearOfJoiningBalvikas',
    'dateOfJoiningBalvikas',
    'hasPassedGroup2Exam',
    'event1',
    'event2',
    'totalParticipatingEvents',
    'arrivalDate',
    'arrivalTime',
    'needsPickup',
    'modeOfTravel',
    'pickupPoint',
    'departureDate',
    'departureTime',
    'needsDrop',
    'modeOfTravelForDrop',
    'dropOffPoint',
    'hasAccompanyingAdults',
    'numMaleAccompanying',
    'numFemaleAccompanying',
    'numNonParticipatingSiblings',
    'accompanyingPersonName',
    'accompanyingPersonGender',
    'accompanyingPersonRelation',
    'accompanyingPersonContact',
    'accompanyingPersonAge',
    'needsAccommodation',
    'numMaleAccompanyingNeedAccommodation',
    'numFemaleAccompanyingNeedAccommodation',
    'foodAllergies',
    'checkInDate',
    'checkInTime',
    'checkOutDate',
    'checkOutTime',
    'overallRegistrationStatus',
    'remarks'
]]

print(len(new_df.columns))

42
42


In [939]:
# Create directory for district wise data
import os

def districtWiseCompleteData(new_df):
    if not os.path.exists("district_wise_complete"):
        os.makedirs("district_wise_complete")
    
    for district in new_df['district'].unique():
        district_df = new_df[new_df['district'] == district]
        # Replace all chars other than alpha numeric with _
        dName = ""
        for c in district:
            if c.isalnum():
                dName += c
            else:
                dName += "_"
        district_df.to_csv(f"district_wise_complete/{dName}.csv", index=False)

def districtWiseEventRegistration(new_df):
    if not os.path.exists("district_wise_event_registration"):
        os.makedirs("district_wise_event_registration")

    for district in new_df['district'].unique():
        district_df = new_df[new_df['district'] == district]
        district_df = district_df[[
            'studentId',
            'studentFullName',
            'dateOfBirth',
            'gender',
            'district',
            'samithiName',
            'event1',
            'event2',
            'overallRegistrationStatus',
            'remarks'
        ]]
        # Replace all chars other than alpha numeric with _
        dName = ""
        for c in district:
            if c.isalnum():
                dName += c
            else:
                dName += "_"
        district_df.to_csv(f"district_wise_event_registration/{dName}_event_registration.csv", index=False)

def eventWiseRegistrationData(new_df):
    if not os.path.exists("event_wise_registration"):
        os.makedirs("event_wise_registration")

    # Collect all events
    events = set()
    for event in new_df['event1'].unique():
        if event != "-":
            events.add(event)
    for event in new_df['event2'].unique():
        if event != "-":
            events.add(event)

    for event in events:
        event_df = new_df[(new_df['event1'] == event) | (new_df['event2'] == event)]
        event_df = event_df[[
            'studentId',
            'studentFullName',
            'dateOfBirth',
            'gender',
            'district',
            'samithiName',
            'event1',
            'event2',
            'overallRegistrationStatus',
            'totalParticipatingEvents',
            'remarks'
        ]]


        if "GROUP" in event:
            # Sort by district
            event_df = event_df.sort_values(by='district')
        else:
            # Sort by totalParticipatingEvents
            event_df = event_df.sort_values(by='totalParticipatingEvents', ascending=False)

        # Replace all chars other than alpha numeric with _
        eName = ""
        for c in event:
            if c.isalnum():
                eName += c
            else:
                eName += "_"

        event_df.to_csv(f"event_wise_registration/{eName}_registration.csv", index=False)

In [940]:
districtWiseCompleteData(new_df)
districtWiseEventRegistration(new_df)
eventWiseRegistrationData(new_df)

In [941]:
group_1 = new_df[new_df['studentGroup'] == 'Group 1']

g1_events = np.unique(np.concatenate((group_1['event1'].unique(), group_1['event2'].unique())))
g1_events = [event for event in g1_events if event != '-']
print(g1_events)
stats = {}

for d in district:
    stats[d] = {}
    for e in g1_events:
        stats[d][e] = len(group_1[(group_1['district'] == d) & ((group_1['event1'] == e) | (group_1['event2'] == e))])
        if stats[d][e] > 1 and ("GROUP" not in e):
            print(d, e, stats[d][e])

['G1-Bhajans', 'G1-Drawing', 'G1-Slokas', 'G1-Story Telling - English', 'G1-Story Telling - Tamizh', 'G1-Tamizh Chants', 'G1-Vedam Chanting', 'GROUP - Devotional Singing - Girls']


In [942]:
group_2 = new_df[new_df['studentGroup'] == 'Group 2']

g2_events = np.unique(np.concatenate((group_2['event1'].unique(), group_2['event2'].unique())))
g2_events = [event for event in g2_events if event != '-']
print(g2_events)
stats = {}

for d in district:
    stats[d] = {}
    for e in g2_events:
        stats[d][e] = len(group_2[(group_2['district'] == d) & ((group_2['event1'] == e) | (group_2['event2'] == e))])
        if stats[d][e] > 1 and ("GROUP" not in e):
            print(d, e, stats[d][e])

['G2-Bhajans - Boys', 'G2-Bhajans - Girls', 'G2-Drawing', 'G2-Elocution - English', 'G2-Elocution - Tamizh', 'G2-Slokas - Boys', 'G2-Slokas - Girls', 'G2-Tamizh Chants - Boys', 'G2-Tamizh Chants - Girls', 'G2-Vedam Chanting - Boys', 'G2-Vedam Chanting - Girls', 'GROUP - Altar Decoration - Boys', 'GROUP - Altar Decoration - Girls', 'GROUP - Devotional Singing - Boys', 'GROUP - Devotional Singing - Girls', 'GROUP - Rudram Namakam Chanting - Boys', 'GROUP - Rudram Namakam Chanting - Girls']


In [943]:
group_3 = new_df[new_df['studentGroup'] == 'Group 3']

g3_events = np.unique(np.concatenate((group_3['event1'].unique(), group_3['event2'].unique())))
g3_events = [event for event in g3_events if event != '-']
print(g3_events)
stats = {}

for d in district:
    stats[d] = {}
    for e in g3_events:
        stats[d][e] = len(group_3[(group_3['district'] == d) & ((group_3['event1'] == e) | (group_3['event2'] == e))])
        if stats[d][e] > 1 and ("GROUP" not in e):
            print(d, e, stats[d][e])

['G3-Bhajans - Boys', 'G3-Bhajans - Girls', 'G3-Drawing', 'G3-Elocution - English', 'G3-Elocution - Tamizh', 'G3-Slokas - Boys', 'G3-Slokas - Girls', 'G3-Tamizh Chants - Boys', 'G3-Tamizh Chants - Girls', 'G3-Vedam Chanting - Boys', 'G3-Vedam Chanting - Girls', 'GROUP - Altar Decoration - Boys', 'GROUP - Altar Decoration - Girls', 'GROUP - Devotional Singing - Boys', 'GROUP - Devotional Singing - Girls', 'GROUP - Rudram Namakam Chanting - Boys', 'GROUP - Rudram Namakam Chanting - Girls', 'Quiz']


In [944]:
group_4 = new_df[new_df['studentGroup'] == 'General Category']

g4_events = np.unique(np.concatenate((group_4['event1'].unique(), group_4['event2'].unique())))
g4_events = [event for event in g4_events if event != '-']
print(g4_events)
stats = {}

for d in district:
    stats[d] = {}
    for e in g4_events:
        stats[d][e] = len(group_4[(group_4['district'] == d) & ((group_4['event1'] == e) | (group_4['event2'] == e))])
        if stats[d][e] > 1 and ("GROUP" not in e):
            print(d, e, stats[d][e])

['Quiz']


In [945]:
g_events = np.unique(np.concatenate((new_df['event1'].unique(), new_df['event2'].unique())))
g_events = [event for event in g_events if event != '-']
# print(g_events)

def process_group_event_data(eventToProcess, allowedCount):
    # Filter events that have "GROUP" in them
    group_events = [event for event in g_events if eventToProcess in event]
    print(group_events)

    stats = {}

    for d in district:
        stats[d] = {}
        for e in group_events:
            stats[d][e] = len(new_df[(new_df['district'] == d) & ((new_df['event1'] == e) | (new_df['event2'] == e))])
            if stats[d][e] > allowedCount:
                print(d, e, stats[d][e])

process_group_event_data('GROUP - Altar Decoration - Boys', 4)

['GROUP - Altar Decoration - Boys']


In [946]:
process_group_event_data('GROUP - Altar Decoration - Girls', 4)

['GROUP - Altar Decoration - Girls']


In [947]:
process_group_event_data('GROUP - Devotional Singing - Boys', 5)

['GROUP - Devotional Singing - Boys']


In [948]:
process_group_event_data('GROUP - Devotional Singing - Girls', 5)

['GROUP - Devotional Singing - Girls']


In [949]:
process_group_event_data('GROUP - Rudram Namakam Chanting - Boys', 4)

['GROUP - Rudram Namakam Chanting - Boys']


In [950]:
process_group_event_data('GROUP - Rudram Namakam Chanting - Girls', 4)

['GROUP - Rudram Namakam Chanting - Girls']


In [951]:
# Sort by district
new_df = new_df.sort_values(by='district')
new_df.to_csv("complete_registration_data.csv", index=False)